<a href="https://colab.research.google.com/github/LeoSkywalker/AtividadeFinal_NovasTecs1/blob/main/AtividadeFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importação de bibliotecas
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Leitura dos dados
dado = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AirPassengers.csv')
print(dado.head())

In [ ]:
# Série Temporal
dado = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AirPassengers.csv', header=0,parse_dates=[0],index_col=[0],squeeze=True)
print(dado.head())

In [ ]:
# Gráfico Série Temporal
plt.title('Passageiros de Transporte Aéreo')
plt.xlabel('Tempo')
plt.ylabel('Número de Passageiros')
plt.plot(dado)

In [ ]:
# Decomposição dos dados
decomposicao = sm.tsa.seasonal_decompose(dado, model='additive', freq=4)
decomposicao.plot()
plt.show()

In [ ]:
# Autocorrelação
plot_acf(dado)
plt.xlabel('Lags')
plt.ylabel('Autocorrelação')
plt.show()

In [ ]:
# Autocorrelação Parcial
plot_pacf(dado)
plt.xlabel('Lags')
plt.ylabel('Autocorrelação Parcial')
plt.show()

In [ ]:
#ARIMA
model = ARIMA(dado, order=(2,1,1))
results = model.fit(disp=0)
print(results.summary())

In [ ]:
# Previsão de Série Previsto-Esperado
size=int(len(dado)*0.75)
train, test = dado[0:size], dado[size:len(dado)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
  model = ARIMA(history, order=(2,1,1))
  model_fit = model.fit(disp=0)
  output = model_fit.forecast()
  yhat = output[0]
  predictions.append(yhat)
  obs = test[t]
  history.append(obs)
  print('Previsto=%f, Esperado=%f' % (yhat, obs))

In [ ]:
# Gráfico de Previsão de Série Temporal
test = np.squeeze(np.array(test))
plt.plot(test)
plt.plot(predictions[1:], color='red')
plt.show()

In [ ]:
# ---- REDES NEURAIS ----
# Gráfico de Séries Temporais com Redes Neurais
plt.plot(dado, label='Air Passengers', color='blue')
plt.show
data=dado.values
np.random.seed(3)

In [ ]:
# Separando dados de treino e teste
data=data.astype('float32')
train = data[0:125]
test = data[125:]

In [ ]:
# Organizando os dados com um lag a frente e separando os dados em originais e passados
def prepare_data(data, lags=1):
  X,Y = [], []
  for row in range(len(data)-lags-1):
    a=data[row:(row+lags)]
    X.append(a)
    Y.append(data[row+lags])
  return np.array(X), np.array(Y)

In [ ]:
# Separação dos dados em originais e passados em treinamento e teste
lags = 1
X_train, Y_train = prepare_data(train, lags)
X_test, Y_test = prepare_data(test, lags)
y_true = Y_test

In [ ]:
# Gráfico de Comparação em Dados Originais e Passados
plt.plot(Y_test, label='Dados Originais', color='purple')
plt.plot(X_test, label='Dados Passados', color='black')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Treinamento da rede
md1 = Sequential()
md1.add(Dense(3, input_dim=lags, activation='relu'))
md1.add(Dense(1))
md1.compile(loss='mean_squared_error', optimizer='adam')
md1.fit(X_train,Y_train,epochs=200,batch_size=2,verbose=2)

In [ ]:
# Gráfico de Comparação de curvas na fase de treinamento
train_predict = md1.predict(X_train)
test_predict = md1.predict(X_test)
plt.plot(X_train, color='purple')
plt.plot(train_predict, color='yellow')
plt.show()

In [ ]:
# Gráfico de Comparação de curvas na fase de teste
plt.plot(X_test, color='purple')
plt.plot(test_predict, color='yellow')
plt.show()

In [ ]:
# Análise de desempenho treinamento-teste (loss)
train_score = print('Treinamento: ', md1.evaluate(X_train, Y_train, verbose=1))
test_score = print('Teste: ', md1.evaluate(X_test, Y_test, verbose=1))
